# RAiDER in Python

RAiDER can be called and used from with Python. 

In [ ]:
# at the most basic level, can "import RAiDER"
import RAiDER
import os

In [ ]:
# Set up a custom logging location
log_dir = './logs/'
if not os.path.exists(log_dir):
    os.mkdir(log_dir)
import RAiDER.cli.conf as conf
conf.LOGGER_PATH = log_dir
from RAiDER.logger import logger

In [ ]:
# we'll need some other basic libraries
import datetime
import pyproj
import xarray
import matplotlib

matplotlib.use('qtagg')

import numpy as np
import matplotlib.pyplot as plt

## Downloading weather model data
RAiDER provides access to multiple weather models through a consistent API. 
RAiDER will process the weather model data to a regular 3D cube and write the data to a NETCDF file that will be used later by the `tropo_delay` function to calculate delays at specified query points. 

In order to download weather model data, we need to instantiate a weather model object of the appropriate class. 

In [ ]:
# We can print out the list of models currently implemented in RAiDER
# Note that some of these models require licenses etc. 
from RAiDER.models.allowed import ALLOWED_MODELS
print(ALLOWED_MODELS)

In [ ]:
# The HRRR model from NOAA can be accessed for free
from RAiDER.models.hrrr import HRRR, HRRRAK

To access weather model data, we need only to specify a datetime and a bounding box in South-North-West-East format

In [ ]:
# First define datetime, and AOI
date_time = datetime.datetime(2018,11,13, 23, 0, 0)

# bounding box is given in SNWE format
ll_bounds = [36.8, 36.85, -76.15, -76.05]

In [ ]:
print(date_time)
print(ll_bounds)

`prepareWeatherModel` is the function for accessing weather model data. 

In [ ]:
from RAiDER.processWM import prepareWeatherModel

In [ ]:
help(prepareWeatherModel)

Instantiate the weather model and then pass it to `prepareWeatherModel`

In [ ]:
weather_model = HRRR()

In [ ]:
# For the first example, we'll do a zenith calculation at the weather model grid nodes
prepareWeatherModel(weather_model, date_time, ll_bounds=ll_bounds, makePlots=True)

You can look at the PDF files generated to see slices of the weather model variables at different heights. 

We can also load the weather model using xarray:

In [ ]:
# we can get the name of the weather model file by passing the write directory to the 'out_file' method
weather_model_file = weather_model.out_file('weather_files')
ds = xarray.load_dataset(weather_model_file)

In [ ]:
ds

In [ ]:
# Plot a slice of the total delay at 500 m height\
plt.close('all')
(ds['wet_total'] + ds['hydro_total']).interp(z=500).plot()
plt.savefig('total_delay.png')

"wet_total" and "hydro_total" are the zenith delays (ZTD) at the weather model grid nodes. 

We can now run the delay calculation using various input query points

### Compare to HRRR-AK

We can compare to using HRRR in Alaska. RAiDER can tell when you want to process data in Alaska versus the Continental US (CONUS), and will automatically switch to the HRRR-AK model for a bounding box within the extent of that model. 

In [ ]:
# bounding box for south-central AK
date_time = datetime.datetime(2018,11,1, 0, 0, 0)
ll_bounds_ak = [60.5, 61.5, -151, -149]

In [ ]:
# instantiate a new weather model
weather_model_ak = HRRRAK()

In [ ]:
prepareWeatherModel(weather_model_ak, date_time, ll_bounds=ll_bounds_ak, makePlots=True)

In [ ]:
weather_model_file_ak = weather_model_ak.out_file('weather_files')
ds_ak = xarray.load_dataset(weather_model_file_ak)
ds_ak

In [ ]:
# Plot a slice of the total delay at 500 m height
plt.close('all')
(ds_ak['wet_total'] + ds_ak['hydro_total']).interp(z=500).plot()
plt.savefig('total_delay_ak.png')

## Delay calculation

The pre-processed weather model already has zenith delays calculated at the grid nodes, but if we want the delays at specific query points we need to use the `tropo_delay` function in RAiDER. 

In [ ]:
from RAiDER.delay import tropo_delay

In [ ]:
help(tropo_delay)

We will need a few more objects for this delay calculation depending on the type of delays desired. 
The `weather_model_file` is the file path to the NETCDF file generated from `prepareWeatherModel`. 

## AOI objects
The AOI object is is one of several types depending on what is requested. 

In [ ]:
# 1. basic data types and input parameters
ll_bounds = [36.8, 36.85, -76.15, -76.05]
from RAiDER.llreader import BoundingBox, StationFile  # also available: RasterRDR

In [ ]:
aoi = BoundingBox(ll_bounds)
print(aoi.bounds())

We can do much more with the AOI objects then just define a bounding box. 
For example, we can load lat/lon files or files containing GNSS station lists: 

In [ ]:
test_aoi = StationFile('data/sample_gnss_list.csv')
print(test_aoi)

Several methods become available upon creation of the object

In [ ]:
test_aoi.bounds()

In [ ]:
test_aoi.projection()

We can use the `readLL` and `readZ` methods to access the lat/lon and elevation values. If no elevation values are available from the data, a DEM will be downloaded on the fly. 

In [ ]:
test_aoi.readLL()

In [ ]:
test_aoi.readZ()

## The LOS object

The LOS object defines the type of delay to calculate; can be zenith delays (ZTD), slant delays (STD) projected from the zenith delays, and slant delays (STD) integrated along the ray path. 

In [ ]:
date_time = datetime.datetime(2018, 11, 13, 0, 0)

In [ ]:
# "Conventional" refers to slant delays by projection
from RAiDER.losreader import Zenith, Conventional, Raytracing

In [ ]:
los = Zenith()

In [ ]:
los.is_Zenith()

If using the `Conventional` or `Raytracing` objects, an orbit file or look vector files should be supplied

In [ ]:
los_ray = Raytracing('data/S1A_OPER_AUX_POEORB_OPOD_20181203T120749_V20181112T225942_20181114T005942.EOF', time=date_time)

In [ ]:
los_ray.is_Zenith()

In [ ]:
los_ray.ray_trace()

### ZTD calculation

Delay calculation in this case will be for a uniform cube at fixed height levels and horizontal spacing. 

We need to setup some objects with the appropriate parameters, including the area of interest (AOI) and line-of-sight (LOS) objects

In [ ]:
weather_model = HRRR()
prepareWeatherModel(weather_model, date_time, ll_bounds=ll_bounds, makePlots=True)

In [ ]:
# Set parameters
aoi.add_buffer(weather_model.getLLRes())
aoi.set_output_xygrid(4326)
los = Zenith()

In [ ]:
# add a buffer determined by latitude for ray tracing
if los.ray_trace():
    wm_bounds = aoi.calc_buffer_ray(
        los.getSensorDirection(),
        lookDir=los.getLookDirection(),
        incAngle=30
    )
else:
    wm_bounds = aoi.bounds()

In [ ]:
weather_model.set_latlon_bounds(wm_bounds, output_spacing=aoi.get_output_spacing())

In [ ]:
# calculate ZTD
ds, _ = tropo_delay(
    date_time,
    weather_model.out_file('weather_files'),
    aoi,
    los,
    height_levels=[0, 100, 500, 1000]
)

Because we asked for a cube, the delays are returned as a single xarray Dataset. In other cases e.g. for rasters, wet and hydrostatic delays will be returned as two ndarrays. 

We can look at the output of the delay calculation in the Dataset

In [ ]:
ds

We can look at the output shape etc.

In [ ]:
print(ds['wet'].shape)
print(ds['wet'].values.mean())
print(ds['wet'].attrs)

Projection information is maintained in the dataset

In [ ]:
print(ds.crs)

In [ ]:
# Plot a slice of the total delay at 500 m height
plt.close('all')
(ds['wet'] + ds['hydro']).interp(z=500).plot()
plt.savefig('ZTD_delays.png')

### Raytracing calculation

In [ ]:
ds, _ = tropo_delay(
    date_time,
    weather_model.out_file('weather_files'),
    aoi,
    los_ray,
    height_levels=[0, 100, 500, 1000]
)

In [ ]:
ds

In [ ]:
# Look at the output
print(ds['wet'].shape)
print(ds['hydro'].shape)
print(ds['wet'].values.mean())
print(ds['hydro'].values.mean())
print(ds['wet'].attrs)

In [ ]:
# Plot a slice of the total delay at 500 m height
plt.close('all')
(ds['wet'] + ds['hydro']).interp(z=500).plot()
plt.savefig('Raytracing_delays.png')